In [3]:
#micro transmissions
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hashlib


def is_pkcs7_padded(message):
    padding = message[-message[-1]:]
    return all(padding[i] == len(padding) for i in range(0, len(padding)))


def decrypt_flag(shared_secret: int, iv: str, ciphertext: str):
    # Derive AES key from shared secret
    sha1 = hashlib.sha1()
    sha1.update(str(shared_secret).encode('ascii'))
    key = sha1.digest()[:16]
    # Decrypt flag
    ciphertext = bytes.fromhex(ciphertext)
    iv = bytes.fromhex(iv)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    plaintext = cipher.decrypt(ciphertext)

    if is_pkcs7_padded(plaintext):
        return unpad(plaintext, 16).decode('ascii')
    else:
        return plaintext.decode('ascii')


shared_secret = secret
iv = "ceb34a8c174d77136455971f08641cc5"
ciphertext = "b503bf04df71cfbd3f464aec2083e9b79c825803a4d4a43697889ad29eb75453"

print(decrypt_flag(shared_secret, iv, ciphertext))


crypto{d0nt_l3t_n_b3_t00_sm4ll}


In [2]:
#micro transmissions
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from random import randint
import hashlib
import time

def bsgs_ecdlp(P, Q, E, m = -1):
    if Q == E((0, 1, 0)):
        return P.order()
    if Q == P:
        return 1
    if m == -1:
        m = ceil(sqrt(P.order()))
    print("order of algo:", m)
    lookup_table = {j*P: j for j in range(m)}
    for i in range(m):
        temp = Q - (i*m)*P
        if temp in lookup_table:
            return (i*m + lookup_table[temp]) % P.order()
    return None

def pohlig_hellman_ec(base,a,phi_p,f):
#solves base^x = a mod p
    l = len(f)
    
    ord = phi_p
    l=[0]*len(f)
    
    for i,(pi,ri) in enumerate(f):
        for j in range(ri):
            c=bsgs(base*(ord//pi),(a-(base*l[i]))*(ord//pi**(j+1)),(0,pi),operation='+')
            l[i] += c*(pi**j)
            
    return crt(l,[pi**ri for pi,ri in f])

def gen_shared_secret(P, n):
    S = n*P
    return S.xy()[0]

# Curve params
p = 99061670249353652702595159229088680425828208953931838069069584252923270946291
a = 1
b = 4

# Define curve
E = EllipticCurve(GF(p), [a, b])

# Protect against Pohlig-Hellman Algorithm
phi = E.order()
f = []
for i in factor(phi):
    if i[0]<210071842937040101:
        f.append(i)

# Create generator
G = E([43190960452218023575787899214023014938926631792651638044680168600989609069200, 20971936269255296908588589778128791635639992476076894152303569022736123671173])

A = E.lift_x(87360200456784002948566700858113190957688355783112995047798140117594305287669)
B = E.lift_x(6082896373499126624029343293750138460137531774473450341235217699497602895121)

start_time = time.time()
n_a = pohlig_hellman_ec(G,A,phi,f)
print("Time Taken for pohlig hellman", time.time()-start_time)

secret = gen_shared_secret(B,n_a)


/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:63: DeprecationWarning: this is being removed from the global namespace
See https://trac.sagemath.org/25785 for details.


Time Taken for pohlig hellman 30.283418655395508
